In [3]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from fbprophet import Prophet
import pycountry
import plotly.express as px
from collections import namedtuple

ModuleNotFoundError: No module named 'pycountry'

In [ ]:
# read csv's
df = pd.read_csv('Covid 19/covid_19_data.csv', parse_dates=['Last Update'])
# rename column names for convenience
df.rename(columns={'ObservationDate':'Date', 'Country/Region':'Country'},inplace=True)
df_confirmed = pd.read_csv('Covid 19/time_series_covid_19_confirmed.csv')
df_recovered = pd.read_csv('Covid 19/time_series_covid_19_recovered.csv')
df_deaths = pd.read_csv('Covid 19/time_series_covid_19_deaths.csv')

df_confirmed.rename(columns={'Country/Region':'Country'}, inplace=True)
df_recovered.rename(columns={'Country/Region':'Country'}, inplace=True)
df_deaths.rename(columns={'Country/Region':'Country'}, inplace=True)

In [ ]:
df_recovered.head()

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
# by date, country view 
d=df.groupby(['Date','Country', 'Province/State']).sum().reset_index()
d


In [ ]:
# Query df by country of cases by last update
df.query('Country=="Mainland China"').groupby('Last Update')[['Confirmed','Deaths','Recovered']].sum().reset_index()

In [ ]:
df.query('Date == "01/22/2020"').groupby('Country')[['Confirmed','Deaths','Recovered']].sum().reset_index()

In [ ]:
df.loc[df['Date']=='01/22/2020']['Country'].unique()

In [ ]:
# Sort by country for overall cases
df.groupby('Country')[['Confirmed', 'Deaths','Recovered']].sum().reset_index()

## Visualizations

In [ ]:
confirmed = df.groupby('Date').sum()['Confirmed'].reset_index()